In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

Reading the raw dataset.

In [ ]:
df = pd.read_csv(open("../raw/qbias.csv"), encoding="utf-8")
df

Dropping useless columns.

In [ ]:
df = df.drop(columns=["Unnamed: 0", "tags", "heading", "source"])

Renaming columns.

In [ ]:
df = df.rename(columns={"text": "body", "bias_rating": "leaning"})

Categorizing the leaning column.

In [ ]:
df["leaning"] = df["leaning"].astype("category")

Stripping the titles and the bodies.

In [ ]:
df["title"] = (df["title"].str.strip()).replace("", np.nan)
df["body"] = (df["body"].str.strip()).replace("", np.nan)

Dropping rows missing either the leaning or both the title and the body.

In [ ]:
df = df.dropna(subset=["leaning"])
df = df.dropna(subset=["title", "body"], how="all")

Printing body duplicates.

In [ ]:
df[df["body"].duplicated(keep=False)]

Dropping the body duplicates.

In [ ]:
df = df.drop_duplicates(subset="body")

Inspecting the title and the body length.

In [ ]:
df["title_length"] = df["title"].str.len()
df["title_word_count"] = df["title"].str.split().str.len()
df["body_length"] = df["body"].str.len()
df["body_word_count"] = df["body"].str.split().str.len()
df = df.sort_values(by="body_length")
df.head()

After the inspection, rows with bodies shorter than 4 words seem to contain no political value. Removing them.

In [ ]:
body_word_count_lower_bound = 4
df = df[df["body_word_count"] >= body_word_count_lower_bound]
# Ensuring to include the last (longest) item.
downsampled = pd.concat([df["body_length"].iloc[::100], df["body_length"].tail(1)]).drop_duplicates()
downsampled.plot.bar().xaxis.set_ticks([]);

In [ ]:
df["title_length"].mean()

In [ ]:
df["body_length"].mean()

The leaning distribution.

In [ ]:
df.groupby("leaning", observed=True).size().plot.pie(autopct="%1.1f%%");

The distribution of body length sums per leaning.

In [ ]:
df.groupby("leaning", observed=True)["body_length"].sum().plot.pie(autopct="%1.1f%%", ylabel="");

The body length distribution by leaning.

In [ ]:
plt.figure(figsize=(10, 6))

for leaning in df["leaning"].unique():
    df_leaning = df[df["leaning"] == leaning]
    downsampled = pd.concat(
        # Ensuring to include the last (longest) item.
        [df_leaning["body_length"].iloc[::100], df_leaning["body_length"].tail(1)]
    ).drop_duplicates().reset_index(drop=True)
    plt.plot(downsampled, label=leaning)

plt.xlabel("downsampled index")
plt.ylabel("body length")
plt.title("body length by political leaning")
plt.legend(title="leaning")
plt.show()

In [ ]:
df.to_parquet("../preprocessed/qbias.parquet")